In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

sys.path.append('../src')
import data_cleaner as dc

In [2]:
RAW_DATA_DIR, PROCESSED_DATA_DIR = dc.caminho_para_dataset()

In [3]:
try:
  df_2022_kaggle = pd.read_csv (PROCESSED_DATA_DIR / 'state_of_data_2022_limpo.csv')
  print('arquivo encontrado')
except FileNotFoundError as e: 
  print("Arquivo nao encontrado " + e)

arquivo encontrado


In [5]:
print(df_2022_kaggle.columns.tolist())

['id_participante', 'idade', 'faixa_idade', 'genero', 'etnia', 'pcd', 'experiencia_prejudicada', "('P1_f ', 'aspectos_prejudicados')", "('P1_g ', 'vive_no_brasil')", "('P1_i ', 'Estado onde mora')", 'uf_residencia', 'regiao_residencia', "('P1_j ', 'Mudou de Estado?')", 'regiao_origem', 'nivel_ensino', 'area_formacao', 'situacao_trabalho', 'setor_atuacao', 'tamanho_empresa', 'atua_como_gestor', "('P2_e ', 'Cargo como Gestor')", 'cargo_atual', 'nivel_hierarquico', 'faixa_salarial', 'experiencia_dados_anos', 'experiencia_ti_anos', 'satisfacao_trabalho', 'motivos_insatisfacao', "('P2_l_1 ', 'Falta de oportunidade de crescimento no emprego atual')", "('P2_l_2 ', 'Salário atual não corresponde ao mercado')", "('P2_l_3 ', 'Não tenho uma boa relação com meu líder/gestor')", "('P2_l_4 ', 'Gostaria de trabalhar em em outra área de atuação')", "('P2_l_5 ', 'Gostaria de receber mais benefícios')", "('P2_l_6 ', 'O clima de trabalho/ambiente não é bom')", "('P2_l_7 ', 'Falta de maturidade analítica 

In [7]:
len(df_2022_kaggle.columns)

357

In [12]:
mapa_renomeacao_2022 = {
  # === Colunas que você já mapeou (mantidas e validadas) ===
  'id_participante': 'id_participante', 'idade': 'idade', 'faixa_idade': 'faixa_idade', 'genero': 'genero', 'etnia': 'etnia', 'pcd': 'pcd', 'experiencia_prejudicada': 'experiencia_prejudicada', 'uf_residencia': 'uf_residencia', 'regiao_residencia': 'regiao_residencia', 'regiao_origem': 'regiao_origem', 'nivel_ensino': 'nivel_ensino', 'area_formacao': 'area_formacao', 'situacao_trabalho': 'situacao_trabalho', 'setor_atuacao': 'setor_atuacao', 'tamanho_empresa': 'tamanho_empresa', 'atua_como_gestor': 'atua_como_gestor', 'cargo_atual': 'cargo_atual', 'nivel_hierarquico': 'nivel_hierarquico', 'faixa_salarial': 'faixa_salarial', 'experiencia_dados_anos': 'experiencia_dados_anos', 'experiencia_ti_anos': 'experiencia_ti_anos', 'satisfacao_trabalho': 'satisfacao_trabalho', 'motivos_insatisfacao': 'motivos_insatisfacao', 'entrevistas_ultimos_6m': 'entrevistas_ultimos_6m', 'planos_mudar_emprego': 'planos_mudar_emprego', 'criterios_escolha_emprego': 'criterios_escolha_emprego', 'modelo_trabalho_atual': 'modelo_trabalho_atual', 'modelo_trabalho_ideal': 'modelo_trabalho_ideal', 'empresa_teve_layoff_2022': 'empresa_teve_layoff', 'tamanho_equipe_dados': 'tamanho_equipe_dados', 'desafios_gestor': 'desafios_gestor', 'linguagens_usadas_dia_a_dia': 'linguagens_usadas_dia_a_dia', 'linguagem_principal': 'linguagem_principal', 'linguagem_preferida': 'linguagem_preferida', 'bancos_dados_usados_dia_a_dia': 'bancos_dados_usados_dia_a_dia', 'clouds_utilizadas': 'clouds_utilizadas', 'cloud_preferida': 'cloud_preferida', 'ferramentas_bi_usadas_dia_a_dia': 'ferramentas_bi_usadas_dia_a_dia', 'tipo_oportunidade_buscada': 'tipo_oportunidade_buscada', 'tempo_busca_emprego_meses': 'tempo_busca_emprego_meses',
  
  # === NOVAS COLUNAS MAPEADAS (COMPLEMENTO) ===
  
  # --- Tema: Experiência Profissional e Demografia ---
  "('P1_f ', 'aspectos_prejudicados')": 'prejudicado_aspectos_lista',
  "('P1_g ', 'vive_no_brasil')": 'vive_no_brasil',
  "('P1_i ', 'Estado onde mora')": 'estado_residencia',
  "('P1_j ', 'Mudou de Estado?')": 'mudou_de_estado',

  # --- Tema: Detalhes do Trabalho ---
  "('P2_e ', 'Cargo como Gestor')": 'cargo_gestor',
  "('P2_r ', 'Caso sua empresa decida pelo modelo 100% presencial qual será sua atitude?')": 'atitude_retorno_presencial',

  # --- Tema: Motivos de Insatisfação e Critérios de Escolha ---
  "('P2_l_1 ', 'Falta de oportunidade de crescimento no emprego atual')": 'insatisfacao_fator_falta_crescimento',
  "('P2_l_2 ', 'Salário atual não corresponde ao mercado')": 'insatisfacao_fator_salario_defasado',
  "('P2_l_3 ', 'Não tenho uma boa relação com meu líder/gestor')": 'insatisfacao_fator_relacao_gestor',
  "('P2_l_4 ', 'Gostaria de trabalhar em em outra área de atuação')": 'insatisfacao_fator_desejo_mudar_area',
  "('P2_l_5 ', 'Gostaria de receber mais benefícios')": 'insatisfacao_fator_beneficios',
  "('P2_l_6 ', 'O clima de trabalho/ambiente não é bom')": 'insatisfacao_fator_ambiente_ruim',
  "('P2_l_7 ', 'Falta de maturidade analítica na empresa')": 'insatisfacao_fator_baixa_maturidade_empresa',
  "('P2_o_1 ', 'Remuneração/Salário')": 'criterio_escolha_remuneracao',
  "('P2_o_2 ', 'Benefícios')": 'criterio_escolha_beneficios',
  "('P2_o_3 ', 'Propósito do trabalho e da empresa')": 'criterio_escolha_proposito',
  "('P2_o_4 ', 'Flexibilidade de trabalho remoto')": 'criterio_escolha_flexibilidade',
  "('P2_o_5 ', 'Ambiente e clima de trabalho')": 'criterio_escolha_ambiente',
  "('P2_o_6 ', 'Oportunidade de aprendizado e trabalhar com referências na área')": 'criterio_escolha_aprendizado',
  "('P2_o_7 ', 'Plano de carreira e oportunidades de crescimento profissional')": 'criterio_escolha_plano_carreira',
  "('P2_o_8 ', 'Maturidade da empresa em termos de tecnologia e dados')": 'criterio_escolha_maturidade_empresa',
  "('P2_o_9 ', 'Qualidade dos gestores e líderes')": 'criterio_escolha_qualidade_gestores',
  "('P2_o_10 ', 'Reputação que a empresa tem no mercado')": 'criterio_escolha_reputacao_empresa',

  # --- Tema: Detalhes da Equipe de Dados e Gestão ---
  "('P3_b ', 'Quais desses papéis/cargos fazem parte do time (ou chapter) de dados da sua empresa?')": 'empresa_cargos_dados_lista',
  "('P3_b_1 ', 'Analytics Engineer')": 'empresa_tem_analytics_engineer',
  "('P3_b_2 ', 'Engenharia de Dados/Data Engineer')": 'empresa_tem_engenheiro_dados',
  "('P3_b_3 ', 'Analista de Dados/Data Analyst')": 'empresa_tem_analista_dados',
  "('P3_b_4 ', 'Cientista de Dados/Data Scientist')": 'empresa_tem_cientista_dados',
  "('P3_b_5 ', 'Database Administrator/DBA')": 'empresa_tem_dba',
  "('P3_b_6 ', 'Analista de Business Intelligence/BI')": 'empresa_tem_analista_bi',
  "('P3_b_7 ', 'Arquiteto de Dados/Data Architect')": 'empresa_tem_arquiteto_dados',
  "('P3_b_8 ', 'Data Product Manager/DPM')": 'empresa_tem_dpm',
  "('P3_b_9 ', 'Business Analyst')": 'empresa_tem_business_analyst',
  "('P3_c ', 'Quais dessas responsabilidades fazem parte da sua rotina atual de trabalho como gestor?')": 'gestor_responsabilidades_lista',
  "('P3_c_1 ', 'Pensar na visão de longo prazo de dados da empresa e fortalecimento da cultura analítica da companhia.')": 'gestor_resp_visao_longo_prazo',
  "('P3_c_2 ', 'Organização de treinamentos e iniciativas com o objetivo de aumentar a maturidade analítica das áreas de negócios.')": 'gestor_resp_treinamentos',
  "('P3_c_3 ', 'Atração, seleção e contratação de talentos para o time de dados.')": 'gestor_resp_contratacao',
  "('P3_c_4 ', 'Decisão sobre contratação de ferramentas e tecnologias relacionadas a dados.')": 'gestor_resp_decisao_ferramentas',
  "('P3_c_5 ', 'Sou gestor da equipe responsável pela engenharia de dados e por manter o Data Lake da empresa como fonte única dos dados, garantindo a qualidade e confiabilidade da informação.')": 'gestor_resp_head_engenharia',
  "('P3_c_6 ', 'Sou gestor da equipe responsável pela entrega de dados, estudos, relatórios e dashboards para as áreas de negócio da empresa.')": 'gestor_resp_head_analise_bi',
  "('P3_c_7 ', 'Sou gestor da equipe responsável por iniciativas e projetos envolvendo Inteligência Artificial e Machine Learning.')": 'gestor_resp_head_ia_ml',
  "('P3_c_8 ', 'Apesar de ser gestor ainda atuo na parte técnica, construindo soluções/análises/modelos etc.')": 'gestor_resp_atua_tecnico',
  "('P3_c_9 ', 'Gestão de projetos de dados, cuidando das etapas, equipes envolvidas, atingimento dos objetivos etc.')": 'gestor_resp_gestao_projetos',
  "('P3_c_10 ', 'Gestão de produtos de dados, cuidando da visão dos produtos, backlog, feedback de usuários etc.')": 'gestor_resp_gestao_produtos',
  "('P3_c_11 ', 'Gestão de pessoas, apoio no desenvolvimento das pessoas, evolução de carreira')": 'gestor_resp_gestao_pessoas',
  "('P3_d_1 ', 'a Contratar novos talentos.')": 'gestor_desafio_contratar_talentos',
  "('P3_d_2 ', 'b Reter talentos.')": 'gestor_desafio_reter_talentos',
  "('P3_d_3 ', 'c Convencer a empresa a aumentar os investimentos na área de dados.')": 'gestor_desafio_obter_investimento',
  "('P3_d_4 ', 'd Gestão de equipes no ambiente remoto.')": 'gestor_desafio_gestao_remota',
  "('P3_d_5 ', 'e Gestão de projetos envolvendo áreas multidisciplinares da empresa.')": 'gestor_desafio_projetos_multiarea',
  "('P3_d_6 ', 'f Organizar as informações e garantir a qualidade e confiabilidade.')": 'gestor_desafio_qualidade_confiabilidade',
  "('P3_d_7 ', 'g Conseguir processar e armazenar um alto volume de dados.')": 'gestor_desafio_volume_dados',
  "('P3_d_8 ', 'h Conseguir gerar valor para as áreas de negócios através de estudos e experimentos.')": 'gestor_desafio_gerar_valor_negocio',
  "('P3_d_9 ', 'i Desenvolver e manter modelos Machine Learning em produção.')": 'gestor_desafio_ml_em_producao',
  "('P3_d_10 ', 'j Gerenciar a expectativa das áreas de negócio em relação as entregas das equipes de dados.')": 'gestor_desafio_gerenciar_expectativas',
  "('P3_d_11 ', 'k Garantir a manutenção dos projetos e modelos em produção, em meio ao crescimento da empresa.')": 'gestor_desafio_manutencao_producao',
  "('P3_d_12 ', 'Conseguir levar inovação para a empresa através dos dados.')": 'gestor_desafio_inovar',
  "('P3_d_13 ', 'Garantir retorno do investimento (ROI) em projetos de dados.')": 'gestor_desafio_garantir_roi',
  "('P3_d_14 ', 'Dividir o tempo entre entregas técnicas e gestão.')": 'gestor_desafio_dividir_tempo_tecnico_gestao',

  # --- Tema: Ferramentas e Tecnologias (Detalhes) ---
  "('P4_a ', 'Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?')": 'atuacao_funcao',
  "('P4_a_1 ', 'Atuacao')": 'atuacao_em_dados',
  "('P4_b ', 'Quais das fontes de dados listadas você já analisou ou processou no trabalho?')": 'fontes_dados_lista',
  "('P4_b_1 ', 'Dados relacionais (estruturados em bancos SQL)')": 'fonte_dados_usa_relacionais',
  "('P4_b_2 ', 'Dados armazenados em bancos NoSQL')": 'fonte_dados_usa_nosql',
  "('P4_b_3 ', 'Imagens')": 'fonte_dados_usa_imagens',
  "('P4_b_4 ', 'Textos/Documentos')": 'fonte_dados_usa_textos',
  "('P4_b_5 ', 'Vídeos')": 'fonte_dados_usa_videos',
  "('P4_b_6 ', 'Áudios')": 'fonte_dados_usa_audios',
  "('P4_b_7 ', 'Planilhas')": 'fonte_dados_usa_planilhas',
  "('P4_b_8 ', 'Dados georeferenciados')": 'fonte_dados_usa_geo',
  "('P4_c ', 'Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?')": 'fonte_dados_principal',
  "('P4_d_1 ', 'SQL')": 'linguagem_usa_sql',
  "('P4_d_2 ', 'R ')": 'linguagem_usa_r',
  "('P4_d_3 ', 'Python')": 'linguagem_usa_python',
  "('P4_d_4 ', 'C/C++/C#')": 'linguagem_usa_c_cplusplus_csharp',
  "('P4_d_5 ', '.NET')": 'linguagem_usa_dotnet',
  "('P4_d_6 ', 'Java')": 'linguagem_usa_java',
  "('P4_d_7 ', 'Julia')": 'linguagem_usa_julia',
  "('P4_d_8 ', 'SAS/Stata')": 'linguagem_usa_sas_stata',
  "('P4_d_9 ', 'Visual Basic/VBA')": 'linguagem_usa_vba',
  "('P4_d_10 ', 'Scala')": 'linguagem_usa_scala',
  "('P4_d_11 ', 'Matlab')": 'linguagem_usa_matlab',
  "('P4_d_12 ', 'PHP')": 'linguagem_usa_php',
  "('P4_d_13 ', 'Javascript')": 'linguagem_usa_javascript',
  "('P4_d_14 ', 'Não utilizo nenhuma linguagem')": 'linguagem_usa_nenhuma',
  "('P4_g_1 ', 'MySQL')": 'banco_dados_usa_mysql',
  "('P4_g_2 ', 'Oracle')": 'banco_dados_usa_oracle',
  "('P4_g_3 ', 'SQL SERVER')": 'banco_dados_usa_sql_server',
  "('P4_f_4 ', 'Amazon Aurora ou RDS')": 'banco_dados_usa_aurora_rds', # Corrigindo inconsistência no nome original
  "('P4_f_5 ', 'DynamoDB')": 'banco_dados_usa_dynamodb',
  "('P4_f_6 ', 'CoachDB')": 'banco_dados_usa_coachdb',
  "('P4_f_7 ', 'Cassandra')": 'banco_dados_usa_cassandra',
  "('P4_f_8 ', 'MongoDB')": 'banco_dados_usa_mongodb',
  "('P4_f_9 ', 'MariaDB')": 'banco_dados_usa_mariadb',
  "('P4_f_10 ', 'Datomic')": 'banco_dados_usa_datomic',
  "('P4_f_11 ', 'S3')": 'banco_dados_usa_s3',
  "('P4_f_12 ', 'PostgreSQL')": 'banco_dados_usa_postgresql',
  "('P4_f_13 ', 'ElasticSearch')": 'banco_dados_usa_elasticsearch',
  "('P4_f_14 ', 'DB2')": 'banco_dados_usa_db2',
  "('P4_f_15 ', 'Microsoft Access')": 'banco_dados_usa_access',
  "('P4_f_16 ', 'SQLite')": 'banco_dados_usa_sqlite',
  "('P4_f_17 ', 'Sybase')": 'banco_dados_usa_sybase',
  "('P4_f_18 ', 'Firebase')": 'banco_dados_usa_firebase',
  "('P4_f_19 ', 'Vertica')": 'banco_dados_usa_vertica',
  "('P4_f_20 ', 'Redis')": 'banco_dados_usa_redis',
  "('P4_f_21 ', 'Neo4J')": 'banco_dados_usa_neo4j',
  "('P4_f_22 ', 'Google BigQuery')": 'banco_dados_usa_bigquery',
  "('P4_f_23 ', 'Google Firestore')": 'banco_dados_usa_firestore',
  "('P4_f_24 ', 'Amazon Redshift')": 'banco_dados_usa_redshift',
  "('P4_f_25 ', 'Amazon Athena')": 'banco_dados_usa_athena',
  "('P4_f_26 ', 'Snowflake')": 'banco_dados_usa_snowflake',
  "('P4_f_27 ', 'Databricks')": 'banco_dados_usa_databricks',
  "('P4_f_28 ', 'HBase')": 'banco_dados_usa_hbase',
  "('P4_f_29 ', 'Presto')": 'banco_dados_usa_presto',
  "('P4_f_30 ', 'Splunk')": 'banco_dados_usa_splunk',
  "('P4_f_31 ', 'SAP HANA')": 'banco_dados_usa_sap_hana',
  "('P4_f_32 ', 'Hive')": 'banco_dados_usa_hive',
  "('P4_f_33 ', 'Firebird')": 'banco_dados_usa_firebird',
  "('P4_h_1 ', 'Azure (Microsoft)')": 'cloud_usa_azure',
  "('P4_h_2 ', 'Amazon Web Services (AWS)')": 'cloud_usa_aws',
  "('P4_h_3 ', 'Google Cloud (GCP)')": 'cloud_usa_gcp',
  "('P4_i_1 ', 'Microsoft PowerBI')": 'bi_usa_powerbi',
  "('P4_i_2 ', 'Qlik View/Qlik Sense')": 'bi_usa_qlik',
  "('P4_i_3 ', 'Tableau')": 'bi_usa_tableau',
  "('P4_i_4 ', 'Metabase')": 'bi_usa_metabase',
  "('P4_i_5 ', 'Superset')": 'bi_usa_superset',
  "('P4_i_6 ', 'Redash')": 'bi_usa_redash',
  "('P4_i_7 ', 'MicroStrategy')": 'bi_usa_microstrategy',
  "('P4_i_8 ', 'IBM Analytics/Cognos')": 'bi_usa_ibm_cognos',
  "('P4_i_9 ', 'SAP Business Objects')": 'bi_usa_sap_bi',
  "('P4_i_10 ', 'Oracle Business Intelligence')": 'bi_usa_oracle_bi',
  "('P4_i_11 ', 'Amazon QuickSight')": 'bi_usa_quicksight',
  "('P4_i_12 ', 'Salesforce/Einstein Analytics')": 'bi_usa_salesforce_einstein',
  "('P4_i_13 ', 'Mode')": 'bi_usa_mode',
  "('P4_i_14 ', 'Alteryx')": 'bi_usa_alteryx',
  "('P4_i_15 ', 'Birst')": 'bi_usa_birst',
  "('P4_i_16 ', 'Looker')": 'bi_usa_looker',
  "('P4_i_17 ', 'Google Data Studio')": 'bi_usa_google_data_studio',
  "('P4_i_18 ', 'SAS Visual Analytics')": 'bi_usa_sas_va',
  "('P4_i_19 ', 'Grafana')": 'bi_usa_grafana',
  "('P4_i_20 ', 'TIBCO Spotfire')": 'bi_usa_tibco',
  "('P4_i_21 ', 'Pentaho')": 'bi_usa_pentaho',
  "('P4_i_22 ', 'Fazemos todas as análises utilizando apenas Excel ou planilhas do google')": 'bi_usa_excel_gsheets',
  "('P4_i_23 ', 'Não utilizo nenhuma ferramenta de BI no trabalho')": 'bi_usa_nenhuma',
  
  # --- Tema: Detalhes sobre Carreiras Específicas ---
  "('P5_a ', 'Qual seu objetivo na área de dados?')": 'objetivo_profissional',
  "('P5_d ', 'Como tem sido a busca por um emprego na área de dados?')": 'carreira_experiencia_processos_seletivos',
  "('P6_a ', 'Quais das opções abaixo fazem parte da sua rotina no trabalho atual como engenheiro de dados?')": 'de_rotina_lista',
  "('P6_a_1 ', 'Desenvolvo pipelines de dados utilizando linguagens de programação como Python, Scala, Java etc.')": 'de_rotina_pipeline_codigo',
  "('P6_a_2 ', 'Realizo construções de ETL's em ferramentas como Pentaho, Talend, Dataflow etc.')": 'de_rotina_etl_ferramentas',
  "('P6_a_3 ', 'Crio consultas através da linguagem SQL para exportar informações e compartilhar com as áreas de negócio.')": 'de_rotina_consultas_sql',
  "('P6_a_4 ', 'Atuo na integração de diferentes fontes de dados através de plataformas proprietárias como Stitch Data, Fivetran etc.')": 'de_rotina_integracao_plataformas',
  "('P6_a_5 ', 'Modelo soluções de arquitetura de dados, criando componentes de ingestão de dados, transformação e recuperação da informação.')": 'de_rotina_modelagem_arquitetura',
  "('P6_a_6 ', 'Desenvolvo/cuido da manutenção de repositórios de dados baseados em streaming de eventos como Data Lakes e Data Lakehouses.')": 'de_rotina_manutencao_datalakes',
  "('P6_a_7 ', 'Atuo na modelagem dos dados, com o objetivo de criar conjuntos de dados como Data Warehouses, Data Marts etc.')": 'de_rotina_modelagem_dados',
  "('P6_a_8 ', 'Cuido da qualidade dos dados, metadados e dicionário de dados.')": 'de_rotina_qualidade_dados',
  "('P6_a_9 ', 'Nenhuma das opções listadas refletem meu dia a dia.')": 'de_rotina_nenhuma_das_opcoes',
  "('P6_b ', 'Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Engineer?')": 'de_ferramentas_etl_lista',
  "('P6_c ', 'Sua organização possui um Data Lake?')": 'de_empresa_possui_data_lake',
  "('P6_d ', 'Qual tecnologia utilizada como plataforma do Data Lake?')": 'de_empresa_tecnologia_data_lake',
  "('P6_e ', 'Sua organização possui um Data Warehouse?')": 'de_empresa_possui_data_warehouse',
  "('P6_f ', 'Qual tecnologia utilizada como plataforma do Data Warehouse?')": 'de_empresa_tecnologia_data_warehouse',
  "('P6_g ', 'Quais as ferramentas de gestão de Qualidade de dados, Metadados e catálogo de dados você utiliza no trabalho?')": 'de_ferramentas_qualidade_dados_lista',
  "('P6_h ', 'Em qual das opções abaixo você gasta a maior parte do seu tempo?')": 'de_maior_tempo_gasto',
  "('P7_1 ', 'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com análise de dados?')": 'da_rotina_lista',
  "('P7_a_1 ', 'Processo e analiso dados utilizando linguagens de programação como Python, R etc.')": 'da_rotina_analise_codigo',
  "('P7_a_2 ', 'Realizo construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik etc.')": 'da_rotina_dashboards_bi',
  "('P7_a_3 ', 'Crio consultas através da linguagem SQL para exportar informações e compartilhar com as áreas de negócio.')": 'da_rotina_consultas_sql',
  "('P7_a_4 ', 'Utilizo API's para extrair dados e complementar minhas análises.')": 'da_rotina_extracao_api',
  "('P7_a_5 ', 'Realizo experimentos e estudos utilizando metodologias estatísticas como teste de hipótese, modelos de regressão etc.')": 'da_rotina_estatistica_testes',
  "('P7_a_6 ', 'Desenvolvo/cuido da manutenção de ETL's utilizando tecnologias como Talend, Pentaho, Airflow, Dataflow etc.')": 'da_rotina_manutencao_etl',
  "('P7_a_7 ', 'Atuo na modelagem dos dados, com o objetivo de criar conjuntos de dados, Data Warehouses, Data Marts etc.')": 'da_rotina_modelagem_dados',
  "('P7_a_8 ', 'Desenvolvo/cuido da manutenção de planilhas para atender as áreas de negócio.')": 'da_rotina_manutencao_planilhas',
  "('P7_a_9 ', 'Utilizo ferramentas avançadas de estatística como SAS')": 'da_rotina_estatistica_avancada',
  "('P7_a_10 ', 'Nenhuma das opções listadas refletem meu dia a dia.')": 'da_rotina_nenhuma_das_opcoes',
  "('P7_b ', 'Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Analyst?')": 'da_ferramentas_etl_lista',
  "('P7_c ', 'Sua empresa utiliza alguma das ferramentas listadas para dar mais autonomia em análise de dados para as áreas de negócio?')": 'da_ferramentas_autonomia_negocios_lista',
  "('P7_d ', 'Em qual das opções abaixo você gasta a maior parte do seu tempo de trabalho?')": 'da_maior_tempo_gasto',
  "('P8_a ', 'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com ciência de dados?')": 'ds_rotina_lista',
  "('P8_a_1 ', 'Estudos Ad-hoc com o objetivo de confirmar hipóteses, realizar modelos preditivos, forecasts, análise de cluster para resolver problemas pontuais e responder perguntas das áreas de negócio.')": 'ds_rotina_estudos_ad_hoc',
  "('P8_a_2 ', 'Sou responsável pela coleta e limpeza dos dados que uso para análise e modelagem.')": 'ds_rotina_coleta_limpeza',
  "('P8_a_3 ', 'Sou responsável por entrar em contato com os times de negócio para definição do problema, identificar a solução e apresentação de resultados.')": 'ds_rotina_contato_negocios',
  "('P8_a_4 ', 'Desenvolvo modelos de Machine Learning com o objetivo de colocar em produção em sistemas (produtos de dados).')": 'ds_rotina_desenvolvimento_ml',
  "('P8_a_5 ', 'Sou responsável por colocar modelos em produção, criar os pipelines de dados, APIs de consumo e monitoramento.')": 'ds_rotina_ml_em_producao',
  "('P8_a_6 ', 'Cuido da manutenção de modelos de Machine Learning já em produção, atuando no monitoramento, ajustes e refatoração quando necessário.')": 'ds_rotina_manutencao_ml',
  "('P8_a_7 ', 'Realizo construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik, etc')": 'ds_rotina_dashboards_bi',
  "('P8_a_8 ', 'Utilizo ferramentas avançadas de estatística como SAS, SPSS, Stata etc, para realizar análises estatísticas e ajustar modelos.Crio e dou manutenção em ETLs, DAGs e automações de pipelines de dados.')": 'ds_rotina_estatistica_avancada',
  "('P8_a_9 ', 'Crio e dou manutenção em ETLs, DAGs e automações de pipelines de dados.')": 'ds_rotina_manutencao_etl',
  "('P8_a_10 ', 'Crio e gerencio soluções de Feature Store e cultura de MLOps.')": 'ds_rotina_feature_store_mlops',
  "('P8_a_11 ', 'Sou responsável por criar e manter a infra que meus modelos e soluções rodam (clusters, servidores, API, containers, etc.)')": 'ds_rotina_infraestrutura',
  "('P8_b ', 'Quais as técnicas e métodos listados abaixo você costuma utilizar no trabalho?')": 'ds_tecnicas_lista',
  "('P8_b_1 ', 'Utilizo modelos de regressão (linear, logística, GLM)')": 'ds_tecnica_regressao',
  "('P8_b_2 ', 'Utilizo redes neurais ou modelos baseados em árvore para criar modelos de classificação')": 'ds_tecnica_classificacao_arvores_redes',
  "('P8_b_3 ', 'Desenvolvo sistemas de recomendação (RecSys)')": 'ds_tecnica_sistemas_recomendacao',
  "('P8_b_4 ', 'Utilizo métodos estatísticos Bayesianos para analisar dados')": 'ds_tecnica_estatistica_bayesiana',
  "('P8_b_5 ', 'Utilizo técnicas de NLP (Natural Language Processing) para análisar dados não-estruturados')": 'ds_tecnica_nlp',
  "('P8_b_6 ', 'Utilizo métodos estatísticos clássicos (Testes de hipótese, análise multivariada, sobrevivência, dados longitudinais, inferência estatistica) para analisar dados')": 'ds_tecnica_estatistica_classica',
  "('P8_b_7 ', 'Utilizo cadeias de Markov ou HMM's para realizar análises de dados')": 'ds_tecnica_markov_hmm',
  "('P8_b_8 ', 'Desenvolvo técnicas de Clusterização (K-means, Spectral, DBScan etc)')": 'ds_tecnica_clusterizacao',
  "('P8_b_9 ', 'Realizo previsões através de modelos de Séries Temporais (Time Series)')": 'ds_tecnica_series_temporais',
  "('P8_b_10 ', 'Utilizo modelos de Reinforcement Learning (aprendizado por reforço)')": 'ds_tecnica_reinforcement_learning',
  "('P8_b_11 ', 'Utilizo modelos de Machine Learning para detecção de fraude')": 'ds_tecnica_deteccao_fraude',
  "('P8_b_l ', 'Utilizo métodos de Visão Computacional')": 'ds_tecnica_visao_computacional',
  "('P8_b_m ', 'Utilizo modelos de Detecção de Churn')": 'ds_tecnica_deteccao_churn',
  "('P8_3 ', 'Quais dessas tecnologias fazem parte do seu dia a dia como cientista de dados?')": 'ds_tecnologias_lista',
  "('P8_c_1 ', 'Ferramentas de BI (PowerBI, Looker, Tableau, Qlik etc)')": 'ds_tecnologia_ferramentas_bi',
  "('P8_c_2 ', 'Planilhas (Excel, Google Sheets etc)')": 'ds_tecnologia_planilhas',
  "('P8_c_3 ', 'Ambientes de desenvolvimento local (R-studio, JupyterLab, Anaconda)')": 'ds_tecnologia_dev_local',
  "('P8_c_4 ', 'Ambientes de desenvolvimento na nuvem (Google Colab, AWS Sagemaker, Kaggle Notebooks etc)')": 'ds_tecnologia_dev_nuvem',
  "('P8_c_5 ', 'Ferramentas de AutoML (Datarobot, H2O, Auto-Keras etc)')": 'ds_tecnologia_automl',
  "('P8_c_6 ', 'Ferramentas de ETL (Apache Airflow, NiFi, Stitch, Fivetran, Pentaho etc)')": 'ds_tecnologia_etl',
  "('P8_c_7 ', 'Plataformas de Machine Learning (TensorFlow, Azure Machine Learning, Kubeflow etc)')": 'ds_tecnologia_plataformas_ml',
  "('P8_c_8 ', 'Feature Store (Feast, Hopsworks, AWS Feature Store, Databricks Feature Store etc)')": 'ds_tecnologia_feature_store',
  "('P8_c_9 ', 'Sistemas de controle de versão (Github, DVC, Neptune, Gitlab etc)')": 'ds_tecnologia_controle_versao',
  "('P8_c_10 ', 'Plataformas de Data Apps (Streamlit, Shiny, Plotly Dash etc)')": 'ds_tecnologia_data_apps',
  "('P8_c_11 ', 'Ferramentas de estatística avançada como SPSS, SAS, etc.')": 'ds_tecnologia_estatistica_avancada',
  "('P8_d ', 'Em qual das opções abaixo você gasta a maior parte do seu tempo no trabalho?')": 'ds_maior_tempo_gasto',


  #restante
  "('P4_c_1 ', 'Dados relacionais (estruturados em bancos SQL)')": 'fonte_dados_principal_relacionais',
  "('P4_c_2 ', 'Dados armazenados em bancos NoSQL')": 'fonte_dados_principal_nosql',
  "('P4_c_3 ', 'Imagens')": 'fonte_dados_principal_imagens',
  "('P4_c_4 ', 'Textos/Documentos')": 'fonte_dados_principal_textos',
  "('P4_c_5 ', 'Vídeos')": 'fonte_dados_principal_videos',
  "('P4_c_6 ', 'Áudios')": 'fonte_dados_principal_audios',
  "('P4_c_7 ', 'Planilhas')": 'fonte_dados_principal_planilhas',
  "('P4_c_8 ', 'Dados georeferenciados')": 'fonte_dados_principal_geo',
  
  # === FERRAMENTAS ETL (Engenheiro de Dados) ===
  "('P6_b_1 ', 'Scripts Python')": 'de_etl_scripts_python',
  "('P6_b_2 ', 'SQL & Stored Procedures')": 'de_etl_sql_procedures',
  "('P6_b_3 ', 'Apache Airflow')": 'de_etl_airflow',
  "('P6_b_4 ', 'Luigi')": 'de_etl_luigi',
  "('P6_b_5 ', 'AWS Glue')": 'de_etl_aws_glue',
  "('P6_b_6 ', 'Talend')": 'de_etl_talend',
  "('P6_b_7 ', 'Pentaho')": 'de_etl_pentaho',
  "('P6_b_8 ', 'Alteryx')": 'de_etl_alteryx',
  "('P6_b_9 ', 'Stitch')": 'de_etl_stitch',
  "('P6_b_10 ', 'Fivetran')": 'de_etl_fivetran',
  "('P6_b_11 ', 'Google Dataflow')": 'de_etl_google_dataflow',
  "('P6_b_12 ', 'Oracle Data Integrator')": 'de_etl_oracle_di',
  "('P6_b_13 ', 'IBM DataStage')": 'de_etl_ibm_datastage',
  "('P6_b_14 ', 'SAP BW ETL')": 'de_etl_sap_bw',
  "('P6_b_15 ', 'SQL Server Integration Services (SSIS)')": 'de_etl_ssis',
  "('P6_b_16 ', 'SAS Data Integration')": 'de_etl_sas_di',
  "('P6_b_17 ', 'Qlik Sense')": 'de_etl_qlik_sense',
  "('P6_b_18 ', 'Knime')": 'de_etl_knime',
  "('P6_b_19 ', 'Databricks')": 'de_etl_databricks',
  "('P6_b_19 ', 'Não utilizo ferramentas de ETL')": 'de_etl_nenhuma', # Nota: Chave duplicada na sua lista, mas mapeada para segurança.

  # === FERRAMENTAS QUALIDADE DE DADOS (Engenheiro de Dados) ===
  "('P6_g_1 ', 'great_expectations')": 'de_qualidade_great_expectations',
  "('P6_g_2 ', 'dbt')": 'de_qualidade_dbt',
  "('P6_g_3 ', 'AWS Deequ')": 'de_qualidade_aws_deequ',
  "('P6_g_4 ', 'Apache Griffin')": 'de_qualidade_apache_griffin',
  "('P6_g_5 ', 'Datafold')": 'de_qualidade_datafold',
  "('P6_g_6 ', 'Amundsen')": 'de_qualidade_amundsen',
  "('P6_g_7 ', 'Monte Carlo')": 'de_qualidade_monte_carlo',
  "('P6_g_8 ', 'SODA')": 'de_qualidade_soda',
  "('P6_g_9 ', 'Big Eye')": 'de_qualidade_big_eye',
  "('P6_g_10 ', 'Data Band')": 'de_qualidade_data_band',
  "('P6_g_11 ', 'Anomalo')": 'de_qualidade_anomalo',
  "('P6_g_l ', 'Metaplane')": 'de_qualidade_metaplane',
  "('P6_g_m ', 'Acceldata')": 'de_qualidade_acceldata',
  
  # === MAIOR TEMPO GASTO (Engenheiro de Dados) ===
  "('P6_h_1 ', 'Desenvolvendo pipelines de dados utilizando linguagens de programação como Python, Scala, Java etc.')": 'de_maior_tempo_pipelines_codigo',
  "('P6_h_2 ', 'Realizando construções de ETL's em ferramentas como Pentaho, Talend, Dataflow etc.')": 'de_maior_tempo_etl_ferramentas',
  "('P6_h_3 ', 'Criando consultas através da linguagem SQL para exportar informações e compartilhar com as áreas de negócio.')": 'de_maior_tempo_consultas_sql',
  "('P6_h_4 ', 'Atuando na integração de diferentes fontes de dados através de plataformas proprietárias como Stitch Data, Fivetran etc.')": 'de_maior_tempo_integracao_plataformas',
  "('P6_h_5 ', 'Modelando soluções de arquitetura de dados, criando componentes de ingestão de dados, transformação e recuperação da informação.')": 'de_maior_tempo_modelagem_arquitetura',
  "('P6_h_6 ', 'Desenvolvendo/cuidando da manutenção de repositórios de dados baseados em streaming de eventos como Data Lakes e Data Lakehouses.')": 'de_maior_tempo_manutencao_datalakes',
  "('P6_h_7 ', 'Atuando na modelagem dos dados, com o objetivo de criar conjuntos de dados como Data Warehouses, Data Marts etc.')": 'de_maior_tempo_modelagem_dados',
  "('P6_h_8 ', 'Cuidando da qualidade dos dados, metadados e dicionário de dados.')": 'de_maior_tempo_qualidade_dados',
  "('P6_h_9 ', 'Nenhuma das opções listadas refletem meu dia a dia.')": 'de_maior_tempo_nenhuma_das_opcoes',

  # === FERRAMENTAS ETL (Analista de Dados) ===
  "('P7_b_1 ', 'Scripts Python')": 'da_etl_scripts_python',
  "('P7_b_2 ', 'SQL & Stored Procedures')": 'da_etl_sql_procedures',
  "('P7_b_3 ', 'Apache Airflow')": 'da_etl_airflow',
  "('P7_b_4 ', 'Luigi')": 'da_etl_luigi',
  "('P7_b_5 ', 'AWS Glue')": 'da_etl_aws_glue',
  "('P7_b_6 ', 'Talend')": 'da_etl_talend',
  "('P7_b_7 ', 'Pentaho')": 'da_etl_pentaho',
  "('P7_b_8 ', 'Alteryx')": 'da_etl_alteryx',
  "('P7_b_9 ', 'Stitch')": 'da_etl_stitch',
  "('P7_b_10 ', 'Fivetran')": 'da_etl_fivetran',
  "('P7_b_11 ', 'Google Dataflow')": 'da_etl_google_dataflow',
  "('P7_b_12 ', 'Oracle Data Integrator')": 'da_etl_oracle_di',
  "('P7_b_13 ', 'IBM DataStage')": 'da_etl_ibm_datastage',
  "('P7_b_14 ', 'SAP BW ETL')": 'da_etl_sap_bw',
  "('P7_b_15 ', 'SQL Server Integration Services (SSIS)')": 'da_etl_ssis',
  "('P7_b_16 ', 'SAS Data Integration')": 'da_etl_sas_di',
  "('P7_b_17 ', 'Qlik Sense')": 'da_etl_qlik_sense',
  "('P7_b_18 ', 'Knime')": 'da_etl_knime',
  "('P7_b_19 ', 'Databricks')": 'da_etl_databricks',
  "('P7_b_20 ', 'Não utilizo ferramentas de ETL')": 'da_etl_nenhuma',

  # === FERRAMENTAS AUTONOMIA NEGÓCIOS (Analista de Dados) ===
  "('P7_c_1 ', 'Ferramentas de AutoML como H2O.ai, Data Robot, BigML etc.')": 'da_autonomia_ferramentas_automl',
  '(\'P7_c_2 \', \'""Point and Click"" Analytics como Alteryx, Knime, Rapidminer etc.\')': 'da_autonomia_ferramentas_point_and_click',
  "('P7_c_3 ', 'Product metricts & Insights como Mixpanel, Amplitude, Adobe Analytics.')": 'da_autonomia_ferramentas_product_insights',
  "('P7_c_4 ', 'Ferramentas de análise dentro de ferramentas de CRM como Salesforce Einstein Anaytics ou Zendesk dashboards.')": 'da_autonomia_ferramentas_crm_analytics',
  "('P7_c_5 ', 'Minha empresa não utiliza essas ferramentas.')": 'da_autonomia_ferramentas_nenhuma',
  "('P7_c_6 ', 'Não sei informar.')": 'da_autonomia_ferramentas_nao_sabe',

  # === MAIOR TEMPO GASTO (Analista de Dados) ===
  "('P7_d_1 ', 'Processando e analisando dados utilizando linguagens de programação como Python, R etc.')": 'da_maior_tempo_analise_codigo',
  "('P7_d_2 ', 'Realizando construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik etc.')": 'da_maior_tempo_dashboards_bi',
  "('P7_d_3 ', 'Criando consultas através da linguagem SQL para exportar informações e compartilhar com as áreas de negócio.')": 'da_maior_tempo_consultas_sql',
  "('P7_d_4 ', 'Utilizando API's para extrair dados e complementar minhas análises.')": 'da_maior_tempo_extracao_api',
  "('P7_d_5 ', 'Realizando experimentos e estudos utilizando metodologias estatísticas como teste de hipótese, modelos de regressão etc.')": 'da_maior_tempo_estatistica_testes',
  "('P7_d_6 ', 'Desenvolvendo/cuidando da manutenção de ETL's utilizando tecnologias como Talend, Pentaho, Airflow, Dataflow etc.')": 'da_maior_tempo_manutencao_etl',
  "('P7_d_7 ', 'Atuando na modelagem dos dados, com o objetivo de criar conjuntos de dados, Data Warehouses, Data Marts etc.')": 'da_maior_tempo_modelagem_dados',
  "('P7_d_8 ', 'Desenvolvendo/cuidando da manutenção de planilhas do Excel ou Google Sheets para atender as áreas de negócio.')": 'da_maior_tempo_manutencao_planilhas',
  "('P7_d_9 ', 'Utilizando ferramentas avançadas de estatística como SAS, SPSS, Stata etc, para realizar análises.')": 'da_maior_tempo_estatistica_avancada',
  "('P7_d_10 ', 'Nenhuma das opções listadas refletem meu dia a dia.')": 'da_maior_tempo_nenhuma_das_opcoes',
  
  # === MAIOR TEMPO GASTO (Cientista de Dados) ===
  "('P8_d_1 ', 'Estudos Ad-hoc com o objetivo de confirmar hipóteses, realizar modelos preditivos, forecasts, análise de cluster para resolver problemas pontuais e responder perguntas das áreas de negócio.')": 'ds_maior_tempo_estudos_ad_hoc',
  "('P8_d_2 ', 'Coletando e limpando os dados que uso para análise e modelagem.')": 'ds_maior_tempo_coleta_limpeza',
  "('P8_d_3 ', 'Entrando em contato com os times de negócio para definição do problema, identificar a solução e apresentação de resultados.')": 'ds_maior_tempo_contato_negocios',
  "('P8_d_4 ', 'Desenvolvendo modelos de Machine Learning com o objetivo de colocar em produção em sistemas (produtos de dados).')": 'ds_maior_tempo_desenvolvimento_ml',
  "('P8_d_5 ', 'Colocando modelos em produção, criando os pipelines de dados, APIs de consumo e monitoramento.')": 'ds_maior_tempo_ml_em_producao',
  "('P8_d_6 ', 'Cuidando da manutenção de modelos de Machine Learning já em produção, atuando no monitoramento, ajustes e refatoração quando necessário.')": 'ds_maior_tempo_manutencao_ml',
  "('P8_d_7 ', 'Realizando construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik, etc.')": 'ds_maior_tempo_dashboards_bi',
  "('P8_d_8 ', 'Utilizando ferramentas avançadas de estatística como SAS, SPSS, Stata etc, para realizar análises.')": 'ds_maior_tempo_estatistica_avancada',
  "('P8_d_9 ', 'Criando e dando manutenção em ETLs, DAGs e automações de pipelines de dados.')": 'ds_maior_tempo_manutencao_etl',
  "('P8_d_10 ', 'Criando e gerenciando soluções de Feature Store e cultura de MLOps.')": 'ds_maior_tempo_feature_store_mlops',
  "('P8_d_11 ', 'Criando e mantendo a infra que meus modelos e soluções rodam (clusters, servidores, API, containers, etc.)')": 'ds_maior_tempo_infraestrutura'
}

In [13]:
len(mapa_renomeacao_2022)

353

In [14]:
df_2022_kaggle_renomeado = df_2022_kaggle.rename(columns=mapa_renomeacao_2022)

In [15]:
print(df_2022_kaggle_renomeado.columns.tolist())

['id_participante', 'idade', 'faixa_idade', 'genero', 'etnia', 'pcd', 'experiencia_prejudicada', 'prejudicado_aspectos_lista', 'vive_no_brasil', 'estado_residencia', 'uf_residencia', 'regiao_residencia', 'mudou_de_estado', 'regiao_origem', 'nivel_ensino', 'area_formacao', 'situacao_trabalho', 'setor_atuacao', 'tamanho_empresa', 'atua_como_gestor', 'cargo_gestor', 'cargo_atual', 'nivel_hierarquico', 'faixa_salarial', 'experiencia_dados_anos', 'experiencia_ti_anos', 'satisfacao_trabalho', 'motivos_insatisfacao', 'insatisfacao_fator_falta_crescimento', 'insatisfacao_fator_salario_defasado', 'insatisfacao_fator_relacao_gestor', 'insatisfacao_fator_desejo_mudar_area', 'insatisfacao_fator_beneficios', 'insatisfacao_fator_ambiente_ruim', 'insatisfacao_fator_baixa_maturidade_empresa', 'entrevistas_ultimos_6m', 'planos_mudar_emprego', 'criterios_escolha_emprego', 'criterio_escolha_remuneracao', 'criterio_escolha_beneficios', 'criterio_escolha_proposito', 'criterio_escolha_flexibilidade', 'crite

In [18]:
print(f'A lista original tem {len(df_2022_kaggle.columns)} colunas')
print(f'Foram renomeadas {len(mapa_renomeacao_2022)} colunas')

A lista original tem 357 colunas
Foram renomeadas 353 colunas


In [21]:
output_path = PROCESSED_DATA_DIR / 'state_of_data_2022_rico_limpo.csv'
df_2022_kaggle_renomeado.to_csv(output_path, index=False)